# Construct a Planner for Agent Orchestration 

An intelligent orchestration planner is designed to seamlessly manage and assign tasks to multiple AI agents. The planner dynamically interprets tasks, evaluates agent capabilities, and optimizes task distribution, ensuring efficient collaboration and effective problem-solving across specialized AI systems.

In [ ]:
import os
import logging
from dotenv import load_dotenv


logging.getLogger('azure.core.pipeline.policies.http_logging_policy').setLevel(
    logging.WARNING)

load_dotenv()
print(os.getenv('CHAT_MODEL'))

# Form the "Cyber Security" Agent Fleet

In [5]:
import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential


project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(), conn_str=os.environ["AIPROJECT_CONNECTION_STRING"]
)

In [ ]:
fleet_name = "internet_threat_analysis"
agent_fleet = []
agent_list = project_client.agents.list_agents().data
for _agent in agent_list:
    if "group" in _agent.metadata.keys() and _agent.metadata["group"] == fleet_name:
        agent_fleet.append({"id": _agent.id, 
                            "name": _agent.name,
                            "description": _agent.description})
        
agent_fleet

# Construct System Prompt for the Planner

In [11]:
aoai_client = project_client.inference.get_azure_openai_client(
    api_version="2024-06-01")

deployment_name = os.environ["CHAT_MODEL"]

## System prompt template

In [7]:
sys_prompt = """You are an intelligent assistant capable of responding to user queries. When you receive a message from the user, you need to determine whether you can answer it directly or if it would be more appropriate to allocate one of the listed agents to respond.

Direct Response: If the question falls within your knowledge and capabilities, you will respond directly. Your response should start with the prefix [__PLANNER__], followed by your answer.

Agent Allocation: If the query is better suited for one of the agents, you will allocate the most appropriate agent based on the user’s message. Your response will begin with the prefix [__AGENT__], followed by the agent's name. If the allocated agent is unable to provide a satisfactory answer, you may decide to allocate a different agent that may better address the user's needs.

Here is the list of agents you will be working with:
[__AGENT_LIST_PLACEHOLDER__]

Make sure to consider the descriptions of each agent when deciding which one to allocate. If an agent cannot find sufficient information or provide a relevant response, you may allocate another agent to ensure the user receives the most relevant and informed answer. Your goal is to provide clear and helpful information to the user, whether through your own knowledge or via the correct agent.

## Constraints
- Do not allocate an agent if you can answer the question directly.
- If there is an agent that can possibly answer the question, allocate that agent.
- If an agent cannot find sufficient information or provide a relevant response, you may allocate another agent to ensure the user receives the most relevant and informed answer.
- If you don't allocate an agent, you must respond with the prefix [__PLANNER__], followed by your response. For example, [__PLANNER__]Hello, I'm here to assist you.;
- If you allocate an agent, you must respond with the prefix [__AGENT__], followed by the agent's name, and then [__TASK__] followed by the request to that agent. For example: [__AGENT__]HR Helpdesk;[__TASK__]Get the company benefit policies.;
- If you think the user's question is fully answered, respond with the prefix [__TERMINATION__] followed by your response.
"""

## Insert Agent Fleet Information to the template

In [ ]:
import json

cyber_fleet_str = json.dumps(agent_fleet).replace("{", '{{').replace("}", '}}')
print(sys_prompt.replace("[__AGENT_LIST_PLACEHOLDER__]", cyber_fleet_str))

## Add Additional Information

In [9]:
addtional_constraint = '''
## Guidelines to generate a report
1. Start with the time keeper to get the current time.
2. Allocate the cyber collector to retrieve latest threat information and the current metrics in use.
3. Use the metric types from the cyber collector to have the db reader fetch database data.
4. Use the CSV files from the db reader to have the data analyzer perform analysis, including forecasting and anomaly detection.
5. Allocate the security evaluator to create radar chart of evaluation metrics.
6. Start report writer to compile information and generate PDF report.
'''

## Form the message list and test the system prompt

### 1. Test the time_keeper agent

In [ ]:
messages = [
    {
        "role": "system",
        "content": sys_prompt.replace("[__AGENT_LIST_PLACEHOLDER__]", cyber_fleet_str) + addtional_constraint
    },
    {
        "role": "user",
        "content": "Generate a cybersecurity report for TrendMicro."
    }
]

response = aoai_client.chat.completions.create(
    model=deployment_name,
    messages=messages,
    temperature=0.7,
    max_tokens=1000
)

print(response.choices[0].message.content)

### 2. Test the cyber_collector agent

In [ ]:
messages = [
    {
        "role": "system",
        "content": sys_prompt.replace("[__AGENT_LIST_PLACEHOLDER__]", cyber_fleet_str) + addtional_constraint
    },
    {
        "role": "user",
        "content": "Generate a cybersecurity report for TrendMicro."
    },
    {
        "role": "assistant",
        "content": "[__AGENT__]time_keeper;[__TASK__]Provide the current time."
    },
    {
        "role": "assistant",
        "content": "[__AGENT__(time_keeper)]The current time is 2025-01-06 19:53:53 (24-hour format)."
    }
]

response = aoai_client.chat.completions.create(
    model=deployment_name,
    messages=messages,
    temperature=0.7,
    max_tokens=1000
)

print(response.choices[0].message.content)

### 3. Test the final agent: report writer

In [ ]:
messages = [
    {
        "role": "system",
        "content": sys_prompt.replace("[__AGENT_LIST_PLACEHOLDER__]", cyber_fleet_str) + addtional_constraint
    },
    {
        "role": "user",
        "content": "Generate a cybersecurity report for TrendMicro."
    },
    {
        "role": "assistant",
        "content": "[__AGENT__]time_keeper;[__TASK__]Provide the current time."
    },
    {
        "role": "assistant",
        "content": "[__AGENT__(time_keeper)]The current time is 2025-01-06 19:53:53 (24-hour format)."
    },
    {
        "role": "assistant",
        "content": "[__AGENT__]cyber_collector;[__TASK__]Collect the latest cybersecurity threat information relevant to TrendMicro."
    },
    {
        "role": "assistant",
        "content": """[__AGENT__(cyber_collector)]Cybersecurity KPIs are essential for measuring the effectiveness of an organization’s security posture. Key metrics include Intrusion Attempts (unauthorized access attempts), Malicious Traffic Proportion (percentage of harmful network activity), Incident Detection Rate (percentage of threats identified), MTTD/MTTR (time to detect/respond to incidents), and Data Breach Volume (amount of compromised data). Other critical KPIs include Patch Compliance (rate of systems updated), False Positive/Negative Rates (accuracy of detection tools), User Awareness Metrics (training effectiveness, e.g., phishing test results), and Vulnerability Management (percentage of critical vulnerabilities remediated). Additional metrics cover Privileged Access Management, Endpoint Protection, Regulatory Compliance, and Cost of Incidents. Tracking these KPIs helps organizations mitigate risks, improve defenses, and ensure compliance.
    Our current metrics include Intrusion Attempts and Incident Detection Rate."""
    },
    {
        "role": "assistant",
        "content": """[__AGENT__]db_reader;[__TASK__]Fetch data for the metrics "Intrusion Attempts" and "Incident Detection Rate" from the database, save as CSV, and provide the file paths."""
    },
    {
        "role": "assistant",
        "content": """[__AGENT__(db_reader)]I have retrieved the requested data and saved it to the specified file paths. Here's the result:
    1. Data for Intrusion Attempts has been saved to: ./data/Intrusion Attempts.csv
    2. Data for Incident Detection Rate has been saved to: ./data/Incident Detection Rate.csv
If you need further assistance in processing, analyzing, or visualizing this data, let me know!"""
    },
    {
        "role": "assistant",
        "content": """[__AGENT__]data_analyzer;[__TASK__]Perform analysis, including forecasting and anomaly detection, on the data in the following files:
1. ./data/Intrusion Attempts.csv
2. ./data/Incident Detection Rate.csv"""
    },
    {
        "role": "assistant",
        "content": """[__AGENT__(Data Analyzer)]After analyzing the data from the provided CSV files, I have generated and reviewed the visualizations for Intrusion Attempts and Incident Detection Rate. Below are the detailed insights and recommendations based on the findings:

# 1. Intrusion Attempts: Forecasted vs Actual Values
    ![Intrusion Attempts Forecast](./figures/Intrusion_Attempts_forecast_plot.png)

    - Trend Analysis: The data reveals a clear upward trend in intrusion attempts over time, indicating a steady increase in malicious activities targeting the system.
    - Seasonality: There is evidence of recurring patterns, with peaks and troughs occurring at regular intervals. This cyclical behavior suggests periodic threats or exploitation of recurring vulnerabilities.
    - Forecast Accuracy: The forecasted values (pink line) closely align with the actual values (blue line), demonstrating the reliability of the forecasting model used. The forecast projects a continued rise in intrusion attempts.

# 2. Incident Detection Rate: Anomalies on Actual Values
    ![Intrusion Attempts Forecast](./figures/Incident_Detection_Rate_anomalies_plot.png)

    - Anomalies Identified: The model detected multiple anomalies (red circles), representing instances where the detection rate significantly deviated from expected behavior. These could indicate system misconfigurations, unusual attack patterns, or delayed responses.
    - Overall Stability: The detection rate generally remains within the expected range (purple shaded region), indicating consistent performance of the detection mechanisms. However, periodic spikes and dips highlight areas for further investigation.
    - Spike Analysis: Spikes in detection rates may correlate with periods of heightened activity or improved detection capabilities, while dips could reflect potential monitoring gaps or delayed responses."""
    },
    {
        "role": "assistant",
        "content": """[__AGENT__]security_evaluator;[__TASK__]Create radar charts for the evaluation metrics "Intrusion Attempts" and "Incident Detection Rate" based on the analyzed data."""
    },
    {
        "role": "assistant",
        "content": """[__AGENT__(security_evaluator)]Let's start by gathering the categories and their corresponding values for your radar chart:\n\n**Categories:**\n1. Vulnerability Score\n2. Detection Rate\n3. Response Time\n4. Threat Intelligence\n5. System Uptime\n\n**Values:**\n- Vulnerability Score: 8\n- Detection Rate: 7\n- Response Time: 6\n- Threat Intelligence: 9\n- System Uptime: 8\n\nNow, I need to confirm that all values are on a scale of 0-10 and that the number of categories matches the number of values. \n\nSince you have five categories, you'll also need to ensure you have five corresponding values, which you do.\n\nI will proceed to generate the radar chart now!"""
    },
    {
        "role": "assistant",
        "content": """[__AGENT__(security_evaluator)]The radar chart has been successfully created! You can find it saved at the following path:\n\n**`./figures/cybersecurity_radar_chart.png`**\n\nThis chart visually represents your organization's cybersecurity metrics, allowing you to analyze areas of strength and those needing improvement. If you have any other questions or need further assistance, feel free to ask!"""
    },
    {
        "role": "assistant",
        "content": """[__AGENT__]report_writer;[__TASK__]Compile all the gathered information, visualizations, and insights into a professional, detailed, and actionable cybersecurity report for TrendMicro. Include the radar chart, analysis, and recommendations. Save the report as a PDF."""
    },
    {
        "role": "assistant",
        "content": """[__AGENT__(report_writer)]The cybersecurity report for TrendMicro has been successfully generated and saved as a PDF. You can download it using the link below:

Download Cybersecurity Threat and Defense Report

If you need any further modifications or additional information, feel free to ask!

"""
    },
]

response = aoai_client.chat.completions.create(
    model=deployment_name,
    messages=messages,
    temperature=0.7,
    max_tokens=1000
)

print(response.choices[0].message.content)